In [1]:
import networkx as nx
import numpy as np

import input_functions as inp
import tsp_routines
from clustering.funcs import k_cluster
from clustering.funcs import best_dropoff
from input_functions.funcs import create_new_graph

In [2]:
def graph_from_input(filename):
    '''graph_from_input(str) --> nx.graph G, np.ndarray[int] homes, dict locToIndex 
    Returns a graph created by reading the input file, with integer vertex labels
    Returns list of the home indices
    Returns a map from integer to the name associated with that node'''
    with open(filename, 'r') as f:
        G = nx.Graph()
        
        locToIndex = {} # maps location name to its index number
        homes = []
        lines = f.readlines()
        
        numLocations = int(lines[0])
        numTAs = int(lines[1])
        locations = lines[2].split()
    
        i = 0
        assert len(locations) == numLocations, "Number of locations must match specified value"
        for loc in locations:
            G.add_node(i)
            locToIndex[loc] = i
            i += 1
            
        TAhomes = lines[3].split()
        assert len(TAhomes) == numTAs, "Number of TA homes must match specified value"
        for home in TAhomes:
            homes.append(locToIndex[home])
        
        homes.insert(0, locToIndex[lines[4].strip()])
        
        row = 0
        for line in lines[5:]:
            line = line.split()
            for col in range(len(line)):
            
                if line[col] != 'x':  
                    G.add_edge(row, col)
                    weight = float(line[col])
                    G[row][col]['weight'] = weight
            row += 1
            
        indexToLoc = {v: k for k, v in locToIndex.items()}
        return G, homes, indexToLoc

In [3]:
def map_dict_entries(input_dict,nodemapper):
    """
    maps the keys and values of the input dict using the nodemapper dict.
    """
    output_dict = dict()
    for key in input_dict.keys():
        mapped_key = nodemapper[key]
        output_dict.update({mapped_key:[]})
        for vertex in input_dict[key]:
            output_dict[mapped_key].append(nodemapper[vertex])
    return output_dict

def add_vertex_to_clusters(clusters,vertex):
    """
    add the given vertex to each cluster.
    Input:
    clusters - dict where the keys are vertices which are cluster centers and the values are a list of 
                vertices belonging to this cluster
    vertex - the vertex to be added to each list in `clusters`
    """
    for key in clusters:
        clusters[key].append(vertex)
        
def get_dropoff_vertices(clusters):
    best_dropoffs = []
    for key in clusters:
        dropoff = best_dropoff(G,clusters[key])
        best_dropoffs.append(dropoff)
    return best_dropoffs
        
def solve_by_clustering(graph,homes,source,num_clusters):
    """
    return the route to be followed by the car as it drops off TAs.
    Inputs:
    graph - input graph
    homes - list of vertices in `graph` that are marked as homes
    source - vertex in `graph` that is the start and end of the path followed by the car
    num_clusters - the number of clusters to be used to group the homes together
    """
    homes_subgraph = tsp_routines.complete_shortest_path_subgraph(graph,homes)
    home_clusters = k_cluster(homes_subgraph,num_clusters)
    # The source vertex is added to each of the clusters before determining the best dropoff location.
    # This is done so that vertices that are closer to the source are given higher preference as dropoff points.
    add_vertex_to_clusters(home_clusters,source)
    dropoff_vertices = get_dropoff_vertices(home_clusters)
    # Add the source to the dropoff vertices
    dropoff_vertices.append(source)
    # Get rid of any repeating entries in the dropoff vertices
    dropoff_vertices = list(set(dropoff_vertices))
    # Construct the fully connected sub-graph with the dropoff vertices 
    # on which TSP is computed
    dropoff_subgraph = tsp_routines.complete_shortest_path_subgraph(graph,dropoff_vertices)
    tsp_route = tsp_routines.metric_mst_tsp(dropoff_subgraph,source)
    return tsp_route
    

In [4]:
G = nx.Graph()
G.add_edge(1,2,weight=1)
G.add_edge(2,3,weight=2)
G.add_edge(2,4,weight=1)
G.add_edge(1,5,weight=2)
G.add_edge(5,6,weight=1)
G.add_edge(5,7,weight=3)
nx.draw(G)

In [35]:
tsp_route = solve_by_clustering(G,homes,homes[0],2)
print(tsp_route)
print(homes)
check_cost(G, tsp_route, homes[1:])

[21, 6, 29, 21]
[21, 47, 19, 46, 24, 7, 23, 31, 37, 32, 42, 34, 36, 1, 27, 2, 11, 17, 25, 9, 6, 13, 29, 8, 10, 41]


(951152593.6022999,
 {47: 6,
  19: 29,
  46: 21,
  24: 6,
  7: 21,
  23: 29,
  31: 6,
  37: 29,
  32: 6,
  42: 6,
  34: 6,
  36: 29,
  1: 6,
  27: 6,
  2: 6,
  11: 21,
  17: 6,
  25: 6,
  9: 21,
  6: 6,
  13: 29,
  29: 29,
  8: 21,
  10: 21,
  41: 21})

In [23]:
def check_cost(G, route, homes):
    carcost = 0
    workingnode = route[0]
    for node in route[1:]:
        carcost+=(2/3)*nx.dijkstra_path_length(G, workingnode, node)
        workingnode = node
        
    ta_cost = 0
    where_get_off = dict()
    for home in homes:
        local_cost = np.inf
        for node in route:
            l = nx.dijkstra_path_length(G, home, node)
            if l < local_cost:
                local_cost = l
                bestnode = node
            else:
                continue
        ta_cost += local_cost
        where_get_off.update({home:bestnode})
    return carcost, where_get_off

In [49]:
def cluster_solver(filename,kiterations = 'all'):
    """returns the best route and its cost and where each ta gets off, found by decreasing nclusters from nhomes to 2"""
    G, homes, node_to_name_map = graph_from_input(filename)
    k = len(homes) #how many clusters initially
    cost = np.inf
    out_route = []
    if kiterations == 'all':
        krange = range(k-2)
    else:
        krange = range(kiterations)
    for i in krange:
        #assumed that homes[0] is the source
        tsp_route = solve_by_clustering(G,homes,homes[0],k)
        tempcost, where_get_off = check_cost(G, tsp_route, homes[1:])
        if tempcost<cost:
            cost = tempcost
            out_route = tsp_route
            dropoffloc_dict = where_get_off
        k-=1
    return out_route, cost, dropoffloc_dict

In [50]:
filename = './inputs/20_50.in'
G, homes, node_to_name_map = graph_from_input(filename)

In [51]:
route, cost, dropoffs = cluster_solver(filename, kiterations='all')

inf
5163591867.49498
5064304275.037486
4974400048.216312
4755108181.049013
4536325976.580492
4445506719.046267
4163429173.991713
3876440371.4332137
3585447008.3529534
3283230284.5359993
2926330038.7796664
2578235644.721813
2408551533.0003533
2070039113.2766333
1940722349.6940198


In [53]:
print(route, cost)

[21, 9, 29, 17, 21] 1308256321.067593
